<a href="https://colab.research.google.com/github/julianox5/Aprendizado-Programacao-Curso-Machine-Learning-do-GoogleDevelopers/blob/master/07_Classificacao_Binaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação Binária
Até agora criamosapenas modelos de regressão. Ou seja, modelos que produziram previsões de ponto flutuante, como "casas neste bairro custam N mil dólares". Neste Colab, vamos criar e avaliar um modelo de classificação binária . Ou seja, criaremos um modelo que responde a uma pergunta binária. Neste exercício, a pergunta binária será: "As casas neste bairro estão acima de determinado preço?"

# Objetivos de aprendizado:
Depois de fazer este Colab, vamos aprender habilidades como:
*   Converter uma pergunta de regressão em uma pergunta de classificação.
*   Modificar o limite de classificação e determine como essa modificação influencia o modelo.
*   Experimentar diferentes métricas de classificação para determinar a eficácia do seu modelo.



## O Conjunto de Dados
Como vários do colabs anteriores, este Colab usa o [conjunto de dados da habitação da california]
(https://developers.google.com/machine-learning/crash-course/california-housing-data-description).

# Sempre usar a versão do tensor flow correta

In [1]:
%tensorflow_version 2.x

# Importar as ferramenta nescessárias para o modelo


In [7]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# ajustar a granularidade do relatorio
pd.options.display.max_rows = 10 
pd.options.display.float_format = "{:.1f}".format
print("importação executada")

importação executada


# Carregando os conjuntos de dados da internet
A célula de código a seguir carrega os arquivos .csv separados e cria os dois DataFrames pandas a seguir:
* `treino_df`, que contém o conjunto de treinamento
* `teste_df`, que contém o conjunto de teste

In [5]:
treino_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
teste_df  = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")
#embaralhar o conjunto de treinamento 
#suffle th training set
treino_df = treino_df.reindex(np.random.permutation(treino_df.index))


Ao contrário de alguns dos Colabs anteriores, a célula de código anterior não dimensionou o rótulo ( median_house_value). A seção a seguir ("Normalizar valores") fornece uma abordagem alternativa.

# Normalizar valores
Ao criar um modelo com vários recursos, os valores de cada recurso devem abranger aproximadamente o mesmo intervalo. Por exemplo, se o intervalo de um recurso se estende por 500 a 100.000 e o de outro recurso se estende por 2 a 12, o modelo será difícil ou impossível de treinar. Portanto, você deve normalizar os recursos em um modelo com vários recursos.

A célula de código a seguir normaliza os conjuntos de dados convertendo cada valor bruto (incluindo o rótulo) em seu Z-score. Um Z-score é o número de desvios padrão da média para um determinado valor bruto. Por exemplo, considere um recurso com as seguintes características:
* Á média é 60
* O Desvio padrão é 10

O valor bruto 75 teria um Z-score de +1,5:
```
Z-score = (75 - 60) / 10 = +1.5
```
O valor bruto 38 teria um escore Z de -2,2:
```
Z-score = (38 - 60) / 10 = -2.2
```


In [ ]:
# Calcular os escores Z de cada coluna no conjunto de treinamento e escrever essas pontuações Z
# em um novo DataFrame do pandas chamado treino_df_norm
treino_df_media = treino_df.mean()
treino_df_dvp   = treino_df.std()
treino_df_norm  = (treino_df - treino_df_media)/ treino_df_dvp

#Examinar alguns do valores do conjunto de treinamento normalizado. 
#Observe que a maioria dos escores Z ficam entre  -2 e +2
treino_df_norm.head()

In [8]:
teste_df_media = teste_df.mean()
teste_df_dvp   = teste_df.std()
teste_df_norm  = (teste_df - teste_df_media) / teste_df_dvp


# Tarefa 1: crie um rótulo binário
Em problemas de classificação, o rótulo para cada exemplo deve ser 0 ou 1. Infelizmente, o rótulo natural no conjunto California Housing `median_house_value`, contém valores de ponto flutuante como 80.100 ou 85.700 em vez de 0s e 1s, enquanto a versão normalizada de `median_house_values` contém flutuante valores de pontos principalmente entre -3 e +3.

Sua tarefa é criar uma nova coluna nomeada `median_house_value_is_high` no conjunto de treinamento e no conjunto de teste. Se median_house_valuefor superior a um determinado valor arbitrário (definido por threshold), defina `median_house_value_is_high` como 1. Caso contrário, defina como 0.

In [ ]:
# Definimos o limite para 265.00, o que é o 75º percentil para valores
# das casas. Com um preço acima de 265.000 será rotulado como 1,
# e todas as outras vizinhanças serão marcadas como 0.
limite = 265000
treino_df_norm["median_house_value_is_high"] = (treino_df["median_house_value"]>limite).astype(float)
treste_df_norm["edian_house_value_is_high"] = (teste_df["median_house_value"]>limite).astype(float)

treino_df_norm["median_house_value_is_high"].head(8000)

